<a href="https://colab.research.google.com/github/ps8012057/projects/blob/master/dog_vs_cat_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
#importing libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, MaxPool2D, Conv2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
import os
import random
import shutil
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import glob
import matplotlib.pyplot as plt

In [2]:
physical_device = tf.config.experimental.list_physical_devices('GPU')
print('num_GPUs_available: ',len(physical_device))
tf.config.experimental.set_memory_growth(physical_device[0], True)

num_GPUs_available:  1


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"princesharma45","key":"b6bc34e22e13d5c057d2bec85ef2b920"}'}

In [4]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#downloading data from kaggle through Api
!kaggle competitions download -c dogs-vs-cats

In [ ]:
#unzip train.zip file
!unzip "train.zip"

In [7]:
#making directiories:
os.chdir('/content/train')
if os.path.isdir('train/dog') is False:
  os.makedirs("train/dog")
  os.makedirs("train/cat")
  os.makedirs("valid/dog")
  os.makedirs("valid/cat")
  os.makedirs("test/dog")
  os.makedirs("test/cat")

In [8]:
#adding data into directories:
for c in random.sample(glob.glob("cat*"), 1000):
  shutil.move(c, "train/cat")
for c in random.sample(glob.glob("dog*"), 1000):
  shutil.move(c, "train/dog")
for c in random.sample(glob.glob("cat*"), 200):
  shutil.move(c, "valid/cat")
for c in random.sample(glob.glob("dog*"), 200):
  shutil.move(c, "valid/dog")
for c in random.sample(glob.glob("cat*"), 100):
  shutil.move(c, "test/cat")
for c in random.sample(glob.glob("dog*"), 100):
  shutil.move(c, "test/dog")

os.chdir('../../')

In [9]:
train_path = '/content/train/train'
test_path = '/content/train/test'
valid_path = '/content/train/valid'

In [10]:
#image preprocessing
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
.flow_from_directory(directory= train_path, target_size= (224,224), classes=["cat","dog"], batch_size= 10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
.flow_from_directory(directory= test_path, target_size= (224,224), classes=["cat","dog"], batch_size= 10, shuffle = False)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
.flow_from_directory(directory= valid_path, target_size= (224,224), classes=["cat","dog"], batch_size= 10)

Found 2000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [17]:
test_batches.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int32)

In [21]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 3s 0us/step


In [22]:
vgg16_model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [23]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [25]:
for layer in model.layers:
  layer.trainable = False

In [26]:
model.add(Dense(units=2, activation="softmax"))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [28]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss= categorical_crossentropy, metrics="accuracy")

In [29]:
model.fit(train_batches, validation_data=valid_batches, epochs=5, verbose=2)

Epoch 1/5
200/200 - 10s - loss: 0.2343 - accuracy: 0.9090 - val_loss: 0.0893 - val_accuracy: 0.9625
Epoch 2/5
200/200 - 10s - loss: 0.0731 - accuracy: 0.9720 - val_loss: 0.0684 - val_accuracy: 0.9725
Epoch 3/5
200/200 - 10s - loss: 0.0524 - accuracy: 0.9825 - val_loss: 0.0642 - val_accuracy: 0.9775
Epoch 4/5
200/200 - 10s - loss: 0.0413 - accuracy: 0.9885 - val_loss: 0.0638 - val_accuracy: 0.9800
Epoch 5/5
200/200 - 10s - loss: 0.0317 - accuracy: 0.9905 - val_loss: 0.0637 - val_accuracy: 0.9825


In [44]:
predictions= model.predict(test_batches, steps=20, verbose=0)

In [ ]:
np.round(predictions)

In [46]:
metrics.confusion_matrix(test_batches.classes,np.argmax(predictions,axis=-1))

array([[95,  5],
       [ 1, 99]])

In [48]:
metrics.f1_score(test_batches.classes,np.argmax(predictions,axis=-1))

0.9705882352941176